<br>

<div align=center><font color=maroon size=6><b>TorchScript</b></font></div>

<font size=4><b>References:</b></font>
* `Docs > `<a href="https://pytorch.org/docs/stable/jit.html" style="text-decoration:none;">TorchScript</a>
* Docs > TorchScript > <a href="https://pytorch.org/docs/stable/jit_language_reference_v2.html" style="text-decoration:none;">TorchScript Language Reference</a>
* Docs > TorchScript > <a href="https://pytorch.org/docs/stable/jit_builtin_functions.html" style="text-decoration:none;">TorchScript Builtins</a>
* Docs > TorchScript > <a href="https://pytorch.org/docs/master/generated/torch.jit.trace.html" style="text-decoration:none;">torch.jit.trace</a>
* Docs > TorchScript > <a href="" style="text-decoration:none;"></a>

<br>
<br>
<br>

<font size=3>TorchScript is a way to create serializable and optimizable models from PyTorch code. Any TorchScript program can be saved from a Python process and loaded in a process where there is no Python dependency.

We provide tools to incrementally transition a model from a pure Python program to a TorchScript program that can be run independently from Python, such as in a standalone C++ program. This makes it possible to train models in PyTorch using familiar tools in Python and then export the model via TorchScript to a production environment where Python programs may be disadvantageous for performance and multi-threading reasons.</font>

<br>

* For a gentle introduction to TorchScript, see the <a href="https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html" style="text-decoration:none;color:maroon;font-size:120%;">Introduction to TorchScript</a> tutorial. <br>
`(详见笔记：D:\KeepStudy\0_Coding\Pytorch\Pytorch_Tutorials\3-2 Introduction to TorchScript.ipynb)`


* For an end-to-end example of converting a PyTorch model to TorchScript and running it in C++, see the <a href="https://pytorch.org/tutorials/advanced/cpp_export.html" style="text-decoration:none;color:maroon;font-size:120%;">Loading a PyTorch Model in C++</a> tutorial. <br>
`(详见笔记：D:\KeepStudy\0_Coding\Pytorch\Pytorch_Tutorials\3-3 Loading a TorchScript Model in C++.ipynb)`

<br>
<br>
<br>

# TorchScript Language Reference <a href="https://pytorch.org/docs/stable/jit_language_reference_v2.html" style="text-decoration:none;font-size:70%">[link]</a>

This reference manual describes the syntax and core semantics of the TorchScript language. 

TorchScript is a statically typed subset of the Python language. 

This document explains the supported features of Python in TorchScript and also how the language diverges from regular Python. 

Any features of Python that are not mentioned in this reference manual are not part of TorchScript. TorchScript focuses specifically on the features of Python that are needed to represent neural network models in PyTorch.

(其它详细内容，详见原网址)

<br>
<br>
<br>

# Creating TorchScript Code <a href="https://pytorch.org/docs/stable/jit.html#creating-torchscript-code" style="text-decoration:none;font-size:70%">[link]</a>

`(详见蓝色 link 链接)`

<br>
<br>
<br>

# Mixing Tracing and Scripting <a href="https://pytorch.org/docs/stable/jit.html#mixing-tracing-and-scripting" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>In many cases either tracing or scripting is an easier approach for converting a model to TorchScript. Tracing and scripting can be composed to suit the particular requirements of a part of a model. </font>

<br>

<font size=3><font color=maroon><b>① Scripted functions can call traced functions</b>. This is particularly useful when you need to use **control-flow** around **a simple feed-forward model**.</font> For instance the beam search of a sequence to sequence model will typically be written in script but can call an encoder module generated using tracing.</font>

Example (calling a traced function in script):

In [1]:
import torch

def foo(x, y):
    return 2 * x + y

traced_foo = torch.jit.trace(foo, (torch.rand(3), torch.rand(3)))


@torch.jit.script
def bar(x):
    return traced_foo(x, x)

In [2]:
bar(torch.tensor(1))

tensor(3)

In [5]:
bar(torch.tensor([1]))

tensor([3])

<br>

<font size=3><font color=maroon><b>② Traced functions can call script functions</b>. This is useful when a small part of a model requires some **control-flow** even though most of the model is just a feed-forward network.</font> Control-flow inside of a script function called by a traced function is preserved correctly.</font>

Example (calling a script function in a traced function):

In [10]:
import torch

@torch.jit.script
def foo(x, y):
    if x.max() > y.max():
        r = x
    else:
        r = y
    return r


def bar(x, y, z):
    return foo(x, y) + z

traced_bar = torch.jit.trace(bar, (torch.rand(3), torch.rand(3), torch.rand(3)))

In [11]:
bar(torch.tensor([1,2]), torch.tensor([2,3]), torch.tensor([2,2]))

tensor([4, 5])

In [13]:
traced_bar(torch.tensor([1,2]), torch.tensor([2,3]), torch.tensor([2,2]))

tensor([4, 5])

<br>

<font size=3>This composition also works for `nn.Modules` as well, where it can be used to generate a submodule using tracing that can be called from the methods of a script module.</font>

Example (using a traced module):

In [1]:
import torch
import torchvision

class MyScriptModule(torch.nn.Module):
    def __init__(self):
        super(MyScriptModule, self).__init__()
        self.means = torch.nn.Parameter(torch.tensor([103.939, 116.779, 123.68])
                                        .resize_(1, 3, 1, 1))
        self.resnet = torch.jit.trace(torchvision.models.resnet18(),
                                      torch.rand(1, 3, 224, 224))

    def forward(self, input):
        return self.resnet(input - self.means)

my_script_module = torch.jit.script(MyScriptModule())

<br>
<br>
<br>

# TorchScript Language <a href="https://pytorch.org/docs/stable/jit.html#torchscript-language" style="text-decoration:none;font-size:70%">[link]</a>

TorchScript is a statically typed subset of Python, so many Python features apply directly to TorchScript. See the full <a href="https://pytorch.org/docs/stable/jit_language_reference.html#language-reference" style="text-decoration:none;color:maroon;font-size:120%;">TorchScript Language Reference</a> for details.

<br>
<br>
<br>

# Built-in Functions and Modules <a href="https://pytorch.org/docs/stable/jit.html#built-in-functions-and-modules" style="text-decoration:none;font-size:70%">[link]</a>

TorchScript supports the use of most PyTorch functions and many Python built-ins. See <a href="https://pytorch.org/docs/stable/jit_builtin_functions.html#builtin-functions" style="text-decoration:none;color:maroon;font-size:120%;">TorchScript Builtins</a> for a full reference of supported functions.

<br>

## PyTorch Functions and Modules <a href="https://pytorch.org/docs/stable/jit.html#pytorch-functions-and-modules" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>TorchScript supports a subset of the tensor and neural network functions that PyTorch provides. Most methods on `Tensor` as well as `functions in the torch namespace`, `all functions in torch.nn.functional` and `most modules from torch.nn` are supported in TorchScript.</font>

See <a href="https://pytorch.org/docs/stable/jit_unsupported.html#jit-unsupported" style="text-decoration:none;color:maroon;font-size:120%;">TorchScript Unsupported Pytorch Constructs</a> for a list of unsupported PyTorch functions and modules.

<br>
<br>

## Python Functions and Modules <a href="https://pytorch.org/docs/stable/jit.html#python-functions-and-modules" style="text-decoration:none;font-size:70%">[link]</a>

Many of Python’s <a href="https://docs.python.org/3/library/functions.html" style="text-decoration:none;font-size:120%;">built-in functions</a> are supported in TorchScript. The <a href="https://docs.python.org/3/library/math.html#module-math" style="text-decoration:none;font-size:120%">math</a> module is also supported (see <a href="https://pytorch.org/docs/stable/jit_builtin_functions.html#math-module" style="text-decoration:none;color:maroon;font-size:120%;">math Module</a> for details), but no other Python modules (built-in or third party) are supported.

<br>
<br>

## Python Language Reference Comparison <a href="https://pytorch.org/docs/stable/jit.html#python-language-reference-comparison" style="text-decoration:none;font-size:70%">[link]</a>

For a full listing of supported Python features, see <a href="https://pytorch.org/docs/stable/jit_python_reference.html#python-language-reference" style="text-decoration:none;color:maroon;font-size:120%;">Python Language Reference Coverage</a>.

<br>

<br>
<br>
<br>

# Debugging <a href="https://pytorch.org/docs/stable/jit.html#debugging" style="text-decoration:none;font-size:70%">[link]</a>

<br>

## Disable JIT for Debugging <a href="https://pytorch.org/docs/stable/jit.html#disable-jit-for-debugging" style="text-decoration:none;font-size:70%">[link]</a>

<br>

<font style="font-size:130%">**`PYTORCH_JIT`**</font>

<font size=3>Setting the environment variable <font size=4>`PYTORCH_JIT=0`</font> will disable all script and tracing annotations. If there is hard-to-debug error in one of your TorchScript models, you can use this flag to force everything to run using native Python. Since TorchScript (scripting and tracing) is disabled with this flag, you can use tools like pdb to debug the model code. For example:</font>

In [2]:
@torch.jit.script
def scripted_fn(x : torch.Tensor):
    for i in range(12):
        x = x + x
    return x

def fn(x):
    x = torch.neg(x)
    
    import pdb; pdb.set_trace()          ### !!!
    
    return scripted_fn(x)

traced_fn = torch.jit.trace(fn, (torch.rand(4, 5),))
traced_fn(torch.rand(3, 4))

> c:\users\18617\appdata\local\temp\ipykernel_1500\2546201766.py(12)fn()

ipdb> 6
6
ipdb> torch.tensor([1,2])


<stdin>:1: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.


tensor([1, 2])
ipdb> exit


BdbQuit: 

<br>

Debugging this script with `pdb` works except for when we invoke the <a href="https://pytorch.org/docs/stable/generated/torch.jit.script.html#torch.jit.script" style="text-decoration:none;font-size:120%">@torch.jit.script</a> function. We can globally disable JIT, so that we can call the `@torch.jit.script` function as a normal Python function and not compile it. 

If the above script is called disable_jit_example.py, we can invoke it like so:

and we will be able to step into the @torch.jit.script function as a normal Python function. <font color=maroon>To disable the TorchScript compiler for a specific function, see <a href="https://pytorch.org/docs/stable/generated/torch.jit.ignore.html#torch.jit.ignore" style="text-decoration:none;font-size:120%">@torch.jit.ignore</a>.</font>

<br>
<br>

## Inspecting Code <a href="https://pytorch.org/docs/stable/jit.html#inspecting-code" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>TorchScript provides a code pretty-printer for all <a href="https://pytorch.org/docs/stable/generated/torch.jit.ScriptModule.html#torch.jit.ScriptModule" style="text-decoration:none;font-size:120%">ScriptModule</a> instances. This pretty-printer gives an interpretation of the script method’s code as valid Python syntax. 

For example:</font>

In [3]:
@torch.jit.script
def foo(len):
    # type: (int) -> torch.Tensor
    rv = torch.zeros(3, 4)
    for i in range(len):
        if i < 10:
            rv = rv - 1.0
        else:
            rv = rv + 1.0
    return rv

print(foo.code)

def foo(len: int) -> Tensor:
  rv = torch.zeros([3, 4])
  rv0 = rv
  for i in range(len):
    if torch.lt(i, 10):
      rv1 = torch.sub(rv0, 1.)
    else:
      rv1 = torch.add(rv0, 1.)
    rv0 = rv1
  return rv0



<font size=3>A `ScriptModule` with a single `forward` method will have an attribute `code`, which you can use to inspect the `ScriptModule’s code`. <font color=maroon>If the ScriptModule has more than one method, you will need to access `.code` on the method itself and not the module.</font> We can inspect the code of a method named `foo` on a ScriptModule by accessing `.foo.code`. The example above produces this output (shown as above):</font>

This is TorchScript’s compilation of the code for the `forward` method. You can use this to ensure TorchScript (tracing or scripting) has captured your model code correctly.

<br>
<br>

## Interpreting Graphs <a href="https://pytorch.org/docs/stable/jit.html#interpreting-graphs" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>TorchScript also has a representation at a lower level than the `code pretty- printer`, in the form of `IR graphs`.

TorchScript uses a `static single assignment (SSA) `**`intermediate representation (IR)`** to represent computation. The instructions in this format consist of ATen (the C++ backend of PyTorch) operators and other primitive operators, including control flow operators for loops and conditionals. </font>

As an example:

In [4]:
@torch.jit.script
def foo(len):
    # type: (int) -> torch.Tensor
    rv = torch.zeros(3, 4)
    for i in range(len):
        if i < 10:
            rv = rv - 1.0
        else:
            rv = rv + 1.0
    return rv

print(foo.graph)

graph(%len.1 : int):
  %21 : int = prim::Constant[value=1]()
  %13 : bool = prim::Constant[value=1]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2108428211.py:5:4
  %5 : NoneType = prim::Constant()
  %1 : int = prim::Constant[value=3]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2108428211.py:4:21
  %2 : int = prim::Constant[value=4]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2108428211.py:4:24
  %16 : int = prim::Constant[value=10]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2108428211.py:6:15
  %20 : float = prim::Constant[value=1.]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2108428211.py:7:22
  %4 : int[] = prim::ListConstruct(%1, %2)
  %rv.1 : Tensor = aten::zeros(%4, %5, %5, %5, %5) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2108428211.py:4:9
  %rv : Tensor = prim::Loop(%len.1, %13, %rv.1) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2108428211.py:5:4
    block0(%i.1 : int, %rv.29 : Tensor):
      %17 : bool = aten::lt(%i.

`graph` follows the same rules described in the <a href="https://pytorch.org/docs/stable/jit.html#inspecting-code" style="text-decoration:none;color:maroon;">Inspecting Code</a> section with regard to `forward` method lookup.

`(From the original site)` The example script above produces the graph:

Take the instruction `%rv.1 : Tensor = aten::zeros(%4, %6, %6, %10, %12) # test.py:9:10` for example.

* `%rv.1 : Tensor` means we assign the output to a (unique) value named `rv.1`, that value is of `Tensor` type and that we do not know its concrete shape.

* `aten::zeros` is the operator (equivalent to `torch.zeros`) and the input list `(%4, %6, %6, %10, %12)` specifies which values in scope should be passed as inputs. The schema for built-in functions like `aten::zeros` can be found at <a href="https://pytorch.org/docs/stable/jit.html#builtin-functions" style="text-decoration:none;color:maroon;font-size:110%;">Builtin Functions</a>.

* `# test.py:9:10` is the location in the original source file that generated this instruction. In this case, it is a file named ***test.py***, on line 9, and at character 10.


Notice that operators can also have associated `blocks`, namely the `prim::Loop` and `prim::If` operators. In the graph print-out, these operators are formatted to reflect their equivalent source code forms to facilitate easy debugging.

<font size=3>Graphs can be inspected as shown to confirm that the computation described by a <a href="https://pytorch.org/docs/stable/generated/torch.jit.ScriptModule.html#torch.jit.ScriptModule" style="text-decoration:none;font-size:120%">ScriptModule</a> is correct, in both automated and manual fashion, as described below.</font>

<br>
<br>

## Tracer <a href="https://pytorch.org/docs/stable/jit.html#tracer" style="text-decoration:none;font-size:70%">[link]</a>

<br>

### Tracing Edge Cases <a href="https://pytorch.org/docs/stable/jit.html#tracing-edge-cases" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>There are some edge cases that exist where the trace of a given Python function/module will not be representative of the underlying code. These cases can include:

* Tracing of control flow that is dependent on inputs (e.g. tensor shapes)

* Tracing of in-place operations of tensor views (e.g. indexing on the left-hand side of an assignment)

Note that these cases may in fact be traceable in the future.</font>

<br>

### Automatic Trace Checking <a href="https://pytorch.org/docs/stable/jit.html#automatic-trace-checking" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>One way to automatically catch many errors in traces is by using <font color=maroon>***check_inputs***</font> on the `torch.jit.trace()` API. `check_inputs` takes a list of tuples of inputs that will be used to re-trace the computation and verify the results.</font> 

For example:

In [5]:
def loop_in_traced_fn(x):
    result = x[0]
    for i in range(x.size(0)):
        result = result * x[i]
    return result

inputs = (torch.rand(3, 4, 5),)
check_inputs = [(torch.rand(4, 5, 6),), (torch.rand(2, 3, 4),)]

traced = torch.jit.trace(loop_in_traced_fn, inputs, check_inputs=check_inputs)

D:\Programs\Anaconda3\envs\ptg\lib\site-packages\torch\jit\_trace.py:799: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 29 / 30 (96.7%)
Greatest absolute difference: 0.2849639654159546 at index (4, 1) (up to 1e-05 allowed)
Greatest relative difference: 12.509584529747938 at index (1, 2) (up to 1e-05 allowed)
  _check_trace(


TracingCheckError: Tracing failed sanity checks!
ERROR: Graphs differed across invocations!
	Graph diff:
		  graph(%x : Tensor):
		    %1 : int = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1571788462.py:2:0
		    %2 : int = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1571788462.py:2:0
		    %result.1 : Tensor = aten::select(%x, %1, %2) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1571788462.py:2:0
		    %4 : int = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1571788462.py:4:0
		    %5 : int = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1571788462.py:4:0
		    %6 : Tensor = aten::select(%x, %4, %5) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1571788462.py:4:0
		    %result.3 : Tensor = aten::mul(%result.1, %6) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1571788462.py:4:0
		    %8 : int = prim::Constant[value=0]() # C:\Users\18617\AppData

<br>

Gives us the following diagnostic information: `(Shown as above or see the original site)`

This message indicates to us that the computation differed between when we first traced it and when we traced it with the `check_inputs`. Indeed, the loop within the body of `loop_in_traced_fn` depends on the shape of the input `x`, and thus when we try another `x` with a different shape, the trace differs.

<font size=3>In this case, data-dependent control flow like this can be captured using <a href="https://pytorch.org/docs/stable/generated/torch.jit.script.html#torch.jit.script" style="text-decoration:none;font-size:120%">torch.jit.script()</a> instead:</font>

In [6]:
def fn(x):
    result = x[0]
    for i in range(x.size(0)):
        result = result * x[i]
    return result

inputs = (torch.rand(3, 4, 5),)
check_inputs = [(torch.rand(4, 5, 6),), (torch.rand(2, 3, 4),)]

scripted_fn = torch.jit.script(fn)
print(scripted_fn.graph)
#print(str(scripted_fn.graph).strip())

for input_tuple in [inputs] + check_inputs:
    torch.testing.assert_close(fn(*input_tuple), scripted_fn(*input_tuple))

graph(%x.1 : Tensor):
  %9 : bool = prim::Constant[value=1]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1970361538.py:3:4
  %2 : int = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1970361538.py:2:15
  %result.1 : Tensor = aten::select(%x.1, %2, %2) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1970361538.py:2:13
  %6 : int = aten::size(%x.1, %2) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1970361538.py:3:19
  %result : Tensor = prim::Loop(%6, %9, %result.1) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1970361538.py:3:4
    block0(%i.1 : int, %result.11 : Tensor):
      %17 : Tensor = aten::select(%x.1, %2, %i.1) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1970361538.py:4:26
      %result.5 : Tensor = aten::mul(%result.11, %17) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/1970361538.py:4:17
      -> (%9, %result.5)
  return (%result)



which produces: `(See original site)`

<br>

### Tracer Warnings <a href="https://pytorch.org/docs/stable/jit.html#tracer-warnings" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>The tracer produces warnings for several problematic patterns in traced computation. As an example, take a trace of a function that contains an in-place assignment on a slice (a view) of a Tensor:</font>

In [7]:
def fill_row_zero(x):
    x[0] = torch.rand(*x.shape[1:2])
    return x

traced = torch.jit.trace(fill_row_zero, (torch.rand(3, 4),))
print(traced.graph)

graph(%x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %4 : int = prim::Constant[value=1]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/485136185.py:2:0
  %5 : int = aten::size(%x, %4) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/485136185.py:2:0
  %6 : Long(device=cpu) = prim::NumToTensor(%5)
  %7 : int = aten::Int(%6)
  %8 : int[] = prim::ListConstruct(%7)
  %9 : int = prim::Constant[value=6]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/485136185.py:2:0
  %10 : NoneType = prim::Constant()
  %11 : Device = prim::Constant[value="cpu"]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/485136185.py:2:0
  %12 : bool = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/485136185.py:2:0
  %13 : Float(4, strides=[1], requires_grad=0, device=cpu) = aten::rand(%8, %9, %10, %11, %12) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/485136185.py:2:0
  %14 : int = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipy

D:\Programs\Anaconda3\envs\ptg\lib\site-packages\torch\jit\_trace.py:810: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%13 : Float(4, strides=[1], requires_grad=0, device=cpu) = aten::rand(%8, %9, %10, %11, %12) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/485136185.py:2:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  _check_trace(
D:\Programs\Anaconda3\envs\ptg\lib\site-packages\torch\jit\_trace.py:810: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4 / 12 (33.3%)
Greatest absolute difference: 0.3016308546066284 at index (0, 3) (up to 1e-05 allowed)
Greatest relative difference: 1.726054506273522 at index (0, 3) (up to 1e-05 allowed)
  _check_trace(


`(From the original site)` Produces several warnings and a graph which simply returns the input:

<br>

<font size=3>We can fix this by modifying the code to not use the in-place update, but rather build up the result tensor out-of-place with `torch.cat`:</font>

In [8]:
def fill_row_zero(x):
    x = torch.cat((torch.rand(1, *x.shape[1:2]), x[1:2]), dim=0)
    return x

traced = torch.jit.trace(fill_row_zero, (torch.rand(3, 4),))
print(traced.graph)

graph(%x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %4 : int = prim::Constant[value=1]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2152967042.py:2:0
  %5 : int = aten::size(%x, %4) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2152967042.py:2:0
  %6 : Long(device=cpu) = prim::NumToTensor(%5)
  %7 : int = aten::Int(%6)
  %8 : int = prim::Constant[value=1]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2152967042.py:2:0
  %9 : int[] = prim::ListConstruct(%8, %7)
  %10 : int = prim::Constant[value=6]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2152967042.py:2:0
  %11 : NoneType = prim::Constant()
  %12 : Device = prim::Constant[value="cpu"]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2152967042.py:2:0
  %13 : bool = prim::Constant[value=0]() # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2152967042.py:2:0
  %14 : Float(1, 4, strides=[4, 1], requires_grad=0, device=cpu) = aten::rand(%9, %10, %11, %12, %13) # C:\Users\18617\AppDa

D:\Programs\Anaconda3\envs\ptg\lib\site-packages\torch\jit\_trace.py:810: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%14 : Float(1, 4, strides=[4, 1], requires_grad=0, device=cpu) = aten::rand(%9, %10, %11, %12, %13) # C:\Users\18617\AppData\Local\Temp/ipykernel_1500/2152967042.py:2:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  _check_trace(
D:\Programs\Anaconda3\envs\ptg\lib\site-packages\torch\jit\_trace.py:810: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4 / 8 (50.0%)
Greatest absolute difference: 0.7674643993377686 at index (0, 2) (up to 1e-05 allowed)
Greatest relative difference: 3.7931623060432633 at index (0, 2) (up to 1e-05 allowed)
  _check_trace(


<br>
<br>
<br>

# Frequently Asked Questions <a href="https://pytorch.org/docs/stable/jit.html#frequently-asked-questions" style="text-decoration:none;font-size:70%">[link]</a>

## <font size=4 color=red>**Q: I would like to train a model on GPU and do inference on CPU. What are the best practices?**</font>

<font size=3>First convert your model from GPU to CPU and then save it, like so:</font>

<font size=4>
    
```python
cpu_model = gpu_model.cpu()
sample_input_cpu = sample_input_gpu.cpu()
traced_cpu = torch.jit.trace(cpu_model, sample_input_cpu)
torch.jit.save(traced_cpu, "cpu.pt")

traced_gpu = torch.jit.trace(gpu_model, sample_input_gpu)
torch.jit.save(traced_gpu, "gpu.pt")

# ... later, when using the model:

if use_gpu:
  model = torch.jit.load("gpu.pt")
else:
  model = torch.jit.load("cpu.pt")

model(input)
```

</font>

<font size=3>This is recommended because the tracer may witness tensor creation on a specific device, so casting an already-loaded model may have unexpected effects. Casting the model before saving it ensures that the tracer has the correct device information.</font>

<br>
<br>

## <font size=4 color=red>**Q: How do I store attributes on a ScriptModule?**</font>

<font size=3>Say we have a model like:</font>

<font size=4>
    
```python
import torch

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.x = 2

    def forward(self):
        return self.x

m = torch.jit.script(Model())
```

</font>

<br>

<font size=3>If `Model` is instantiated it will result in a compilation error since the compiler doesn’t know about `x`. There are 4 ways to inform the compiler of attributes on `ScriptModule`:
<br>
<br>
    1. <font color=maroon>**nn.Parameter**</font> - Values wrapped in `nn.Parameter` will work as they do on `nn.Modules`
<br>
<br>
    2. <font color=maroon>**register_buffer**</font> - Values wrapped in `register_buffer` will work as they do on `nn.Modules`. This is equivalent to an attribute (see 4) of type `Tensor`.
<br>
<br>
    3. <font color=maroon>**Constants**</font> - Annotating a class member as `Final` (or adding it to a list called `__constants__` at the class definition level) will mark the contained names as constants. Constants are saved directly in the code of the model. See ***builtin-constants*** for details.
<br>
<br>
    4. <font color=maroon>**Attributes**</font> - Values that are a ***supported type*** can be added as mutable attributes. Most types can be inferred but some may need to be specified, see ***module attributes*** for details.
</font>

<br>
<br>

## <font size=4 color=red>**Q: I would like to trace module’s method but I keep getting this error:**</font>

**`RuntimeError: Cannot insert a Tensor that requires grad as a constant. Consider making it a parameter or input, or detaching the gradient`**

<font size=3>This error usually means that the method you are tracing uses a module’s parameters and you are passing the module’s method instead of the module instance (e.g. `my_module_instance.forward` vs `my_module_instance`).

* Invoking `trace` with a module’s method captures module parameters (which may require gradients) as **constants**.
<br>
<br>
* On the other hand, invoking `trace` with module’s instance (e.g. `my_module`) creates a new module and correctly copies parameters into the new module, so they can accumulate gradients if required.


To trace a specific method on a module, see <a href="https://pytorch.org/docs/stable/generated/torch.jit.trace_module.html#torch.jit.trace_module" style="text-decoration:none;font-size:120%">torch.jit.trace_module</a>
</font>

<br>
<br>
<br>

# Known Issues <a href="https://pytorch.org/docs/stable/jit.html#known-issues" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>If you’re using `Sequential` with TorchScript, the inputs of some of the `Sequential` submodules may be falsely inferred to be `Tensor`, even if they’re annotated otherwise. <font color=maroon>The canonical solution is to subclass `nn.Sequential` and redeclare `forward` with the input typed correctly.</font></font>

<br>

<br>
<br>
<br>

# Appendix <a href="https://pytorch.org/docs/stable/jit.html#appendix" style="text-decoration:none;font-size:70%">[link]</a>

## Migrating to PyTorch 1.2 Recursive Scripting API <a href="https://pytorch.org/docs/stable/jit.html#migrating-to-pytorch-1-2-recursive-scripting-api" style="text-decoration:none;font-size:70%">[link]</a>

`(详见蓝色 link 链接的原网址内容)`

<br>
<br>
<br>

# References <a href="https://pytorch.org/docs/stable/jit.html#references" style="text-decoration:none;font-size:70%">[link]</a>

`(详见蓝色 link 链接的原网址内容)`

<br>
<br>
<br>

<br>
<br>
<br>